# Generating pitch tracks for the Guše visualizer

In [ ]:
import json
import os
import numpy as np
import os
import matplotlib.pyplot as plt

In [ ]:
folderName = '../files/pitchTracks/'
mbid = '66a75b20-87a4-4ee8-b1b5-5d1d96749db5'
pitchTrackExtension = '.f0_fil.csv'
fileName = mbid + pitchTrackExtension

dur = 56
shahed = 227.61

filePath = os.path.join(folderName, fileName)
print(filePath)

### From Hz to cents

In [ ]:
def h2c(t, h):
    return np.rint(1200 * np.log2(h/t))

In [ ]:
# Load csv
pitchTrackHz = np.genfromtxt(filePath, delimiter=",")
pitchTrackHz

In [ ]:
maxCent = h2c(shahed, np.max(pitchTrackHz[:,1]))
print("Max:", maxCent)

minCent = h2c(shahed, np.min(pitchTrackHz[:,1]))
print("Min:", minCent)

In [ ]:
maxCent = 500
# minCent = -722

minCut = True

In [ ]:
# Prepare dic
pitchTrackDic = {}

for frame in pitchTrackHz:
    pitchTrackDic["{:.2f}".format(frame[0])] = frame[1]

In [ ]:
# Find duration

if pitchTrackHz[-1,0] > dur:
    dur = pitchTrackHz[-1,0]

dur

In [ ]:
# Cent pitch track

pitchTrackCents = {}

for i in range(dur * 100):
    k = "{:.2f}".format(i/100)
    if k not in pitchTrackDic.keys():
        pitchTrackCents[k] = "s"
    else:
        c = h2c(shahed, pitchTrackDic[k])
        if minCut:
            if c >= minCent and c <= maxCent:
                pitchTrackCents[k] = c
        else:
            pitchTrackCents[k] = c

pitchTrackCents

In [ ]:
with open(folderName + mbid + "_pitchTrack.json", 'w') as f:
    json.dump(pitchTrackCents, f)

In [ ]:
x = 0
y = 0
for k in pitchTrackCents.keys():
    if pitchTrackCents[k] != 's' and pitchTrackCents[k] > x:
        x = pitchTrackCents[k]
    if pitchTrackCents[k] != 's' and pitchTrackCents[k] < y:
        y = pitchTrackCents[k]
print(y, x)

### Pitch histogram

In [ ]:
import intonation
import numpy as np
import json
import os

In [ ]:
def pitch2cents(pitch, ref):
    '''
    Converts a pitch track in Hz to cents given a reference pitch
    
    Args:
        pitch (np.array): a numpy array with two columns, first for time,
                          second for pitch
        ref (float): the reference pitch
    
    Returns:
        cents (np.array) a numpy array with one column of pitch in cents
    '''

    return 1200 * np.log2(pitch[:,1] / float(ref))

In [ ]:
# folderName = './'
# mbid = '7d88c651-5dea-48e5-b21a-ac964cff5cad'
# pitchTrackExtension = '.f0_fil.csv'

# shahed = 332.67

In [ ]:
pitchTrack = np.genfromtxt(os.path.join(folderName, mbid + pitchTrackExtension), delimiter=',')
centsTrack = pitch2cents(pitchTrack, shahed)
filteredTrack = centsTrack[np.logical_and(centsTrack >= minCent, centsTrack <= maxCent)]
pitch_obj = intonation.Pitch(np.arange(len(filteredTrack)), filteredTrack)
# pitch_obj = intonation.Pitch(np.arange(len(centsTrack)), centsTrack)
rec_obj = intonation.Recording(pitch_obj)
rec_obj.compute_hist()

print(rec_obj.histogram.x)
print(rec_obj.histogram.y)

In [ ]:
histDic = {}

for i in range(rec_obj.histogram.x.size):
    # pos = '{:.2f}'.format(rec_obj.histogram.x[i])
    x = round(rec_obj.histogram.x[i], 2)
    y = round(rec_obj.histogram.y[i] * 10000, 2)
    histDic[x] = y

histMin = np.min(list(histDic.values()))
histMax = np.max(list(histDic.values()))

print(histMin, histMax)

histList = []
for k, v in histDic.items():
    histList.append([k,v])

hist2json = {'min': histMin, 'max': histMax, 'hist': histList}
hist2json

In [ ]:
pitchHistogramFolder = '../files/pitchHistograms'
pitchHistogramExtension = '_pitchHistogram.json'

json.dumps(hist2json)

with open(os.path.join(pitchHistogramFolder, mbid + pitchHistogramExtension), 'w') as f:
    json.dump(hist2json, f)